<a href="https://colab.research.google.com/github/Hataeju/Detection/blob/main/detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비

## 기본 라이브러리 Import

In [1]:
# 기본라이브러리 import
from google.colab import drive
import os, json, requests, time  #VirusTotal API결과 분석을 위해

import numpy as np
import pandas as pd


## 라이브러리 추가 설치 및 Import

In [2]:
!pip install pefile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 2.2 MB/s eta 0:00:00


In [3]:
!pip install yara-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.5 MB/s eta 0:00:00


In [ ]:
# hashlib은 이제 내장되어있다/
#!pip install\ hashlib

In [4]:
# 추가 라이브러리 import
import hashlib
import pefile
import yara

## 데이터 로드

In [34]:
# 구글 드라이브 mount
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
# 데이터 파일 위치
colab_path = "gdrive/My Drive/AI보안교육/part2/data/"


# 메타데이터 분석

## 메타데이터 - Hash

In [36]:
# DataFrame 초기화
feature_dict = {'hash':[], 'target':[]}

# Hash를 구하기 위해 hashlib 라이브러리 사용
for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb') #바이너리 모드로 읽기
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        feature_dict['hash'].append(hash)
        if dir == "normal":
            feature_dict['target'].append(0)
        else:
            feature_dict['target'].append(1)
df = pd.DataFrame.from_dict(feature_dict)
df = df.set_index('hash')

In [37]:
df

,target
hash,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0


In [38]:
# DataFrame 초기화
feature_dict = {"filename":[], 'md5':[], 'sha256':[], 'sha512':[], 'target':[]}

# Hash를 구하기 위해 hashlib 라이브러리 사용
for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb') #바이너리 모드로 읽기
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        md5 = hashlib.md5(data).hexdigest()
        sha256 = hashlib.sha256(data).hexdigest()
        sha512 = hashlib.sha512(data).hexdigest()
        feature_dict['filename'].append(filename)
        feature_dict['md5'].append(md5)
        feature_dict['sha256'].append(sha256)
        feature_dict['sha512'].append(sha512)
        if dir == "normal":
            feature_dict['target'].append(0)
        else:
            feature_dict['target'].append(1)
df2 = pd.DataFrame.from_dict(feature_dict)
df2 = df2.set_index('filename')

In [39]:
df2

,md5,sha256,sha512,target
filename,,,,
cmd.exe,9d59442313565c2e0860b88bf32b2277,d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b...,c489ccb79a65ec41800db7e95d8e4993d7780488355869...,1
net.exe,a63df9a6e9098cc189f2a3efc37600f6,96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17c...,38fb182b83a4d069bc2f3773f6a1b103fce7f0b9d6b6d5...,1
NETSTAT.EXE,2445dece99deedbd701dc6dfe10e648e,5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5...,5815bea5548b47515ce4df76b1cd9618f71a8307aafc7e...,1
systeminfo.exe,4e678d82dde61aaaca201f9636507f9e,f5b4fb293ada97a8c4b92bb380747da1d71336f144332c...,395bf5a1069ac4da7893524337e8e90b4367392c58ded4...,1
powershell.exe,cda48fc75952ad12d99e526d0b6bf70a,908b64b1971a979c7e3e8ce4621945cba84854cb98d763...,31946a5c80088646fe951081e88a7478b804da874236a4...,1
notepad.exe,f1139811bbf61362915958806ad30211,f1d62648ef915d85cb4fc140359e925395d315c70f3566...,5c13fdb1e973d146f6bc33083a023d18bb54d075d7fded...,0
SnippingTool.exe,d57ca86ac22dc057456acc7fde4e492f,6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f...,c0c1145e229d73b939df48b69f7eff80ea3cabb8292349...,0
mspaint.exe,33a61f754d9433d6cc37388ce2711040,f1e3d8851c7b41e3838eae69a95224c92abd9a33117238...,8fe75d720a438ab215f22b5ee873e4476ddd7d9758d2e0...,0
calc.exe,5da8c98136d98dfec4716edd79c7145f,58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba...,6e2b067760ec178cdcc4df04c541ce6940fc2a0cdd36f5...,0


## 메타데이터 - size/name/엔트로피

In [ ]:
from scipy import stats

def get_entropy(filename, start, end):
    f = open(filename, 'rb') #바이트 단위로 read
    information = []
    for i in range(end-start):
        s = f.read(1)
        # 1바이트씩(0~255) 읽어서 정수로 변환: 같은 byte의 발생 frequency 계산을 위해
        information.append(int.from_bytes(s, 'little'))
    value, counts = np.unique(information, return_counts=True)
    # 0~255 각 값의 발생확률을 scipy.stats.entropy를 통해 엔트로피 계산
    probs = counts/sum(counts)
    result = stats.entropy(probs, base=2)
    return result

In [ ]:
# 최종적으로는 파일을 한 번 Load하여 관련 Feature를 전부 추출하겠지만,
# 여기에서는 이론과 병행 학습을 위해 매번 파일을 로드하여 Feature를 하나씩 추가하는 방식을 택한다.

df['size'] = None
df['filename'] = None
df['entropy'] = None

for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 등 정보 기록
        size = os.path.getsize(file_path + filename)
        df.loc[hash, 'size'] = size
        df.loc[hash, 'filename'] = filename
        entropy=get_entropy(file_path + filename, 0, size)
        df.loc[hash, 'entropy'] = entropy




In [ ]:
df

,target,size,filename,entropy
hash,,,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,280064,cmd.exe,6.16622
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,57856,net.exe,5.79086
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,38400,NETSTAT.EXE,5.575415
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,102912,systeminfo.exe,5.702098
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,451584,powershell.exe,5.453382
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,181248,notepad.exe,6.379497
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,3379712,SnippingTool.exe,4.351111
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,947712,mspaint.exe,6.158041
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,27648,calc.exe,3.911298


# PE Information

## PE Information - Image File Characteristics

In [ ]:
# 구조체 정보 확인
target_file = colab_path + "normal/calc.exe"
pe = pefile.PE(target_file)
hex(pe.FILE_HEADER.Characteristics)

'0x22'

In [ ]:
# Feature로 등록
df['IFC'] = None

for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 등 정보 기록
        pe = pefile.PE(file_path + filename)
        ifc = hex(pe.FILE_HEADER.Characteristics)
        df.loc[hash, 'IFC'] = ifc



In [ ]:
df

,target,size,filename,entropy,IFC
hash,,,,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,280064,cmd.exe,6.16622,0x22
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,57856,net.exe,5.79086,0x22
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,38400,NETSTAT.EXE,5.575415,0x22
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,102912,systeminfo.exe,5.702098,0x22
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,451584,powershell.exe,5.453382,0x22
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,181248,notepad.exe,6.379497,0x22
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,3379712,SnippingTool.exe,4.351111,0x22
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,947712,mspaint.exe,6.158041,0x22
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,27648,calc.exe,3.911298,0x22


## EntryPoint, ImageBase

In [ ]:
# 구조체 정보 확인
target_file = colab_path + "normal/calc.exe"
pe = pefile.PE(target_file)
print(hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint))
print(hex(pe.OPTIONAL_HEADER.ImageBase))

0x1870
0x140000000


## 섹션 엔트로피 / 사이즈

In [ ]:
# 구조체 정보 확인
target_file = colab_path + "normal/calc.exe"
pe = pefile.PE(target_file)
for section in pe.sections:
    if (section.Name).decode('ascii').rstrip('\x00') == '.text':
        start = section.PointerToRawData
        end = section.Misc_VirtualSize
    if (section.Name).decode('ascii').rstrip('\x00') == '.rsrc':
        section_size = section.SizeOfRawData
pe.close()
with open(target_file, 'rb') as f:
    f.seek(start)
    text_data = f.read(end)
text_data

b'\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc@SH\x83\xec@L\x8b\x94$\x80\x00\x00\x00H\x8b\xd9M\x85\xd2t\\L\x8b\\$p\x8b\xc2\x85\xd2t#\x83\xf8\x01u"A\x0f\xb6\xc8\xb8\x00\x01\x00\x00\xff\xc1M\x89J\x10E\x84\xc0M\x89Z\x18\x0fE\xc1A\x89\x02\xeb\x04A\x83"\x00I\x8bB(H\x85\xc0t!I\x8bJ0H\x89L$0H\x8bL$xH\x89L$(H\x8b\xcbL\x89\\$ \xff\x15\xee\x11\x00\x00H\x83\xc4@[\xc3\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xccH\x83\xecH\x0f\xb6\x02L\x8dR\x01H\x8bT$xL\x8b\xd9\xc1\xe0\x18\x89D$0A\x0f\xb7\x02\x89D$4I\x8bB\x02I\x83\xc2\nH\x89D$8H\x8bA\x08H\x89\x02H\x8bA\x08H\x89T$(\x0f\xb7\x08\x89J\x08H\x8d\r\x9c\x12\x00\x00\xc7B\x0c\x02\x00\x00\x00L\x89R\x10A\x0f\xb7\x02\x89B\x18H\x8d\x05\x03\x13\x00\x00+\xc1\xc7B\x1c\x01\x00\x00\x00I\x8bK H\x8dT$0\x89D$x\x8bD$p\x89D$ H\xff\x15\x16\x10\x00\x00\x0f\x1fD\x00\x00H\x83\xc4H\xc3\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xccf\x90\xc3\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xcc\xccH\x81\xec\x88\x00\x00\x00H\x8b\x05\xf6\x1e\x00\x00H3\xc4H\x89D$pH\x8b\x05\xaf\x1e\x00\x00L\x8d\

In [ ]:
# Feature로 등록
df['ts_entropy'] = np.nan
df['rs_size'] = None
df['entrypoint'] = None
df['imagebase'] = None

for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 등 정보 기록
        pe = pefile.PE(file_path + filename)

        for section in pe.sections:
            if (section.Name).decode('ascii').rstrip('\x00') == '.text':
                start = section.PointerToRawData
                end = section.Misc_VirtualSize
            if (section.Name).decode('ascii').rstrip('\x00') == '.rsrc':
                section_size = section.SizeOfRawData
        pe.close()
        section_entropy = get_entropy(file_path + filename, start, end)
        df.loc[hash, 'ts_entropy'] = section_entropy
        df.loc[hash, 'rs_size'] = section_size
        df.loc[hash, 'entrypoint'] = hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint)
        df.loc[hash, 'imagebase'] = hex(pe.OPTIONAL_HEADER.ImageBase)





In [ ]:
df

,target,size,filename,entropy,IFC,ts_entropy,rs_size,entrypoint,imagebase
hash,,,,,,,,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,280064,cmd.exe,6.16622,0x22,6.375660,34304,0x18160,0x140000000
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,57856,net.exe,5.79086,0x22,6.194608,2048,0x2750,0x140000000
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,38400,NETSTAT.EXE,5.575415,0x22,6.146966,2560,0x6310,0x140000000
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,102912,systeminfo.exe,5.702098,0x22,6.033320,2560,0xff70,0x140000000
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,451584,powershell.exe,5.453382,0x22,6.123446,359936,0x3b50,0x140000000
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,181248,notepad.exe,6.379497,0x22,6.370246,3072,0x20100,0x140000000
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,3379712,SnippingTool.exe,4.351111,0x22,6.374814,3100160,0x315b0,0x140000000
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,947712,mspaint.exe,6.158041,0x22,6.330385,3584,0x960a0,0x140000000
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,27648,calc.exe,3.911298,0x22,4.617768,18432,0x1870,0x140000000


# Static Info

## Yara 기본 동작 원리

In [ ]:
# Text String 기반 탐지
rule = yara.compile(source='rule connection_msg {strings: $a = "connect" condition: $a}')  #C2통신등에 공격자 콘솔에 표시되는 문자열 탐지
matches = rule.match(data='connecting to XXX:')
matches

[connection_msg]

In [ ]:
# 외부 rule import를 통한 탐지
rule_file = "gdrive/My Drive/AI보안교육/part2/yara_rules/sample.yar"
yara_rule = open(rule_file, 'r').read()
rule = yara.compile(source=yara_rule)
matches = rule.match(data='connecting to XXX:')  # 메세지 바꿔서 탐지 안되는 것 역시 확인
matches

[connection_msg]

In [ ]:
# Feature로 등록
df['yara_https'] = None
df['yara_ipaddr'] = None
rule1 = yara.compile(source='rule ip_addr {strings: $a = /(\d{1,3}\.){3}\d{1,3}/ condition: $a}')  #IP address
rule2 = yara.compile(source='rule https {strings: $a = /https:\S+/ condition: $a}') #https
for dir in os.listdir(colab_path):
    file_path = colab_path + dir + '/'
    for filename in os.listdir(file_path):
        f = open(file_path + filename, 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 등 정보 기록
        matches = rule1.match(data=data)
        if len(matches) != 0:
            df.loc[hash, 'yara_ipaddr'] = True
        else:
            df.loc[hash, 'yara_ipaddr'] = False
        matches = rule2.match(data=data)
        if len(matches) != 0:
            df.loc[hash, 'yara_https'] = True
        else:
            df.loc[hash, 'yara_https'] = False



In [ ]:
df

,target,size,filename,entropy,IFC,ts_entropy,rs_size,entrypoint,imagebase,yara_https,yara_ipaddr
hash,,,,,,,,,,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,280064,cmd.exe,6.16622,0x22,6.375660,34304,0x18160,0x140000000,False,True
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,57856,net.exe,5.79086,0x22,6.194608,2048,0x2750,0x140000000,False,True
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,38400,NETSTAT.EXE,5.575415,0x22,6.146966,2560,0x6310,0x140000000,False,True
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,102912,systeminfo.exe,5.702098,0x22,6.033320,2560,0xff70,0x140000000,False,True
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,451584,powershell.exe,5.453382,0x22,6.123446,359936,0x3b50,0x140000000,False,False
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,181248,notepad.exe,6.379497,0x22,6.370246,3072,0x20100,0x140000000,False,True
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,3379712,SnippingTool.exe,4.351111,0x22,6.374814,3100160,0x315b0,0x140000000,False,True
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,947712,mspaint.exe,6.158041,0x22,6.330385,3584,0x960a0,0x140000000,False,True
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,27648,calc.exe,3.911298,0x22,4.617768,18432,0x1870,0x140000000,False,True


# Dynamic Info

## VT API 기본 사용법

In [ ]:
from google.colab import userdata
apikey = userdata.get('VT_API_KEY')

In [ ]:
# Trickbot 샘플로 행위분석 결과 정보 파악

#apikey = ""   # 자신의 API Key
hash = "32c58040d3d6ec5305a1a0ebb48ba05aebe3ac2f905a7f152f32fc9170e16711" #SHA-256, SHA-1 또는 MD5

url = "https://www.virustotal.com/api/v3/files/" + hash + "/behaviours?limit=40"
headers = {"Accept": "application/json", "x-apikey": apikey}

response = requests.get(url, headers=headers)

In [ ]:
vt_json = response.json()
#vt_json

In [ ]:
# prompt: vt_json을 colab_path 밑에 덤프 하고 싶어

#with open(os.path.join(colab_path, 'vt_results.json'), 'w') as f:
    #json.dump(vt_json, f)

In [ ]:
colab_path

'gdrive/My Drive/AI보안교육/part2/data/'

In [ ]:
# json 최상위 구조 파악
vt_json.keys()

dict_keys(['data', 'links', 'meta'])

In [ ]:
# 결과를 제공한 sandbox 벤더의 수
vt_json['meta']

{'count': 12,
 'sandboxes_in_progress': ['Zenbox'],
 'in_progress_percent': {'Zenbox': 71},
 'sandbox_status': {'Zenbox': {'in_progress_percent': 71,
   'status': 'running'}}}

In [ ]:
# 쿼리 링크
vt_json['links']

{'self': 'https://www.virustotal.com/api/v3/files/32c58040d3d6ec5305a1a0ebb48ba05aebe3ac2f905a7f152f32fc9170e16711/behaviours?limit=40'}

In [ ]:
len(vt_json['data'])

12

In [ ]:
type(vt_json['data'][0])

dict

In [ ]:
# 각 벤더마다 동일한 구조로 결과 제공 (여기서는 0~8까지 인덱스. 총 9개 벤더사 결과이므로)
vt_json['data'][0].keys()

dict_keys(['id', 'type', 'links', 'attributes'])

In [ ]:
# sandbox 벤더
#vt_json['data'][0]['id']
#vt_json['data'][0]['attributes']['sandbox_name']


In [ ]:
# 분석 결과물 구조 파악
# 각 벤더마다 제공되는 정보가 모두 다르다. (만약 데이터가 존재한다면 JSON 키값은 동일하다.)
print("{}의 제공정보: {}".format(vt_json['data'][0]['id'].split('_')[1], vt_json['data'][0]['attributes'].keys()))
print("{}의 제공정보: {}".format(vt_json['data'][1]['id'].split('_')[1], vt_json['data'][1]['attributes'].keys()))

CAPA의 제공정보: dict_keys(['has_pcap', 'signature_matches', 'has_memdump', 'behash', 'sandbox_name', 'last_modification_date', 'has_html_report', 'mbc', 'analysis_date', 'mitre_attack_techniques', 'has_evtx'])
CAPE Sandbox의 제공정보: dict_keys(['has_pcap', 'processes_tree', 'files_opened', 'has_memdump', 'behash', 'files_dropped', 'registry_keys_opened', 'mutexes_created', 'sandbox_name', 'last_modification_date', 'has_html_report', 'calls_highlighted', 'registry_keys_set', 'analysis_date', 'mitre_attack_techniques', 'modules_loaded', 'command_executions', 'has_evtx', 'services_opened'])


In [ ]:
# 그 중 하나의 정보 추출
vt_json['data'][4]['attributes']['files_deleted']

['%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_yqwude4r.ymc.ps1',
 '%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_hmukj3es.y4n.psm1',
 '%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_pyc24mut.05y.ps1',
 '%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_5yoxlx1a.ene.psm1',
 'C:\\Windows\\System32\\spp\\store\\2.0\\cache\\cache.dat',
 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER3275.tmp.WERInternalMetadata.xml',
 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER342A.tmp.csv',
 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER346A.tmp.txt',
 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER5FCE.tmp.WERInternalMetadata.xml',
 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER5FE0.tmp.csv',
 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER5FF0.tmp.txt',
 '%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_et0pwu2x.nks.ps1',
 '%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_sx344yeh.kvv.psm1',
 '%USERPROFILE

## Dynamic Info - File Activities

In [8]:
# VT JSON 파일에서 File관련된 Activity만 확인
hash = '908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53'  # 첫번째 샘플에 대해서
load_path = "gdrive/My Drive/AI보안교육/part2/vt_json/" + hash + ".json"
with open(load_path) as f:
   vt_json = json.load(f)

In [9]:
vt_json['meta']['count']

3

In [14]:
# files_deleted 의 데이터 파악
for vendor_idx in range(vt_json['meta']['count']): #해당 샘플의 경우 vendor 1개
    print("====={}=====".format(vt_json['data'][vendor_idx]['id'].split('_')[1]))
    if 'attributes' in vt_json['data'][vendor_idx].keys():
        for (key, value) in (vt_json['data'][vendor_idx]['attributes'].items()):
            if key in ['files_deleted']:
                print(key, value)


=====Microsoft Sysinternals=====
files_deleted ['%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_jowriwtf.fuh.ps1', '%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_ga5wgcm5.0yu.psm1', 'C:\\Windows\\System32\\spp\\store\\2.0\\cache\\cache.dat', '%USERPROFILE%\\AppData\\Local\\Microsoft\\Windows\\PowerShell\\ModuleAnalysisCache', 'C:\\Windows\\ServiceProfiles\\LocalService\\AppData\\Roaming\\Microsoft\\Crypto\\Keys\\de7cf8a7901d2ad13e5c67c29e5d1662_cbbb49d6-b7ff-44ca-aba5-8a5e250d4d42', '%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_hvekdxfu.egc.ps1', '%USERPROFILE%\\AppData\\Local\\Temp\\__PSScriptPolicyTest_fqfnxd3m.uzb.psm1', 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER569.tmp.WERInternalMetadata.xml', 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER56A.tmp.WERInternalMetadata.xml', 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER635.tmp.csv', 'C:\\ProgramData\\Microsoft\\Windows\\WER\\Temp\\WER665.tmp.txt', 'C:\\ProgramData\\Microsoft\

In [40]:
# 삭제된 파일의 카운트 (각 동적분석 벤더 분석 결과의 평균값).
df['file_deleted_cnt'] = np.nan

for dir in os.listdir(colab_path):
    file_path = colab_path + dir +'/'
    for filename in os.listdir(file_path):
        valid_vendor_cnt = 0
        file_deleted_cnt_total = 0

        f = open(file_path + filename , 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 정보 기록

        load_path = "gdrive/My Drive/AI보안교육/part2/vt_json/" + hash + ".json"
        with open(load_path) as f:
            vt_json = json.load(f)
        for vendor_idx in range(vt_json['meta']['count']): #해당 샘플의 경우 vendor 1개
            file_deleted_cnt = 0
            for(key, value) in (vt_json['data'][vendor_idx]['attributes'].items()):
                if key in ['files_deleted']:
                    file_deleted_cnt = len(value)
                    valid_vendor_cnt += 1
                    file_deleted_cnt_total += file_deleted_cnt
        if valid_vendor_cnt != 0:
            df.loc[hash, 'file_deleted_cnt'] = file_deleted_cnt_total/valid_vendor_cnt

In [41]:
df

,target,file_deleted_cnt
hash,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,31.0
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,62.0
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,10.0
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,7.0
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,190.0
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,NaN
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,7.0
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,NaN
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,81.0


In [ ]:
# 삭제된 파일의 카운트 (각 동적분석 벤더 분석 결과의 평균값).
df['file_created_cnt'] = np.nan

for dir in os.listdir(colab_path):
    file_path = colab_path + dir +'/'
    for filename in os.listdir(file_path):
        valid_vendor_cnt = 0
        valid_add_vendor = set()
        file_created_cnt_total = 0

        f = open(file_path + filename , 'rb')
        data = f.read()
        hash = hashlib.sha256(data).hexdigest()
        # hash값 기준으로 size 정보 기록

        #load_path = "gdrive/My Drive/AI보안교육/part2/vt_json/" + hash + ".json"
        #with open(load_path) as f:
        #    vt_json = json.load(f)
        for vendor_idx in range(vt_json['meta']['count']): #해당 샘플의 경우 vendor 1개
            file_created_cnt = 0
            for(key, value) in (vt_json['data'][vendor_idx]['attributes'].items()):
                if key in ['files_dropped']:
                    file_created_cnt = len(value)
                    valid_vendor_cnt += 1
                    file_created_cnt_total += file_created_cnt
                if key in ['files_written']:
                    file_created_cnt = len(value)
                    valid_vendor_cnt += 1
                    file_created_cnt_total += file_created_cnt
                if key in ['files_copied']:
                    file_created_cnt = len(value)
                    valid_vendor_cnt += 1
                    file_created_cnt_total += file_created_cnt


        if valid_vendor_cnt != 0:
            df.loc[hash, 'file_created_cnt'] = file_created_cnt_total/valid_vendor_cnt


In [ ]:
df

,target,file_deleted_cnt,file_created_cnt
hash,,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,27.4,33.090909
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,27.4,33.090909
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,27.4,33.090909
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,27.4,33.090909
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,27.4,33.090909
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,27.4,33.090909
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,27.4,33.090909
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,27.4,33.090909
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,27.4,33.090909


## Dynamic Info - Registry / Process Activities (실습)

In [42]:
for vendor_idx in range(vt_json['meta']['count']):
    attrs = vt_json['data'][vendor_idx]['attributes']
    # 속성 이름 중에 'registry' 가 들어가는 것만 골라본다
    registry_keys = [k for k in attrs.keys() if 'registry' in k.lower()]
    print("==== 벤더 {}: {} ====".format(
        vt_json['data'][vendor_idx]['id'].split('_')[1], registry_keys))

==== 벤더 Microsoft Sysinternals: [] ====
==== 벤더 ReaQta-Hive: ['registry_keys_set'] ====
==== 벤더 Zenbox: ['registry_keys_opened', 'registry_keys_set'] ====


In [43]:
import os, json, hashlib, numpy as np

# 1) 결과를 담을 컬럼 추가
df['registry_open_cnt'] = np.nan
df['registry_set_cnt'] = np.nan

for dir_name in os.listdir(colab_path):
    folder = os.path.join(colab_path, dir_name)
    for filename in os.listdir(folder):
        # 파일 해시 계산
        file_path = os.path.join(folder, filename)
        with open(file_path, 'rb') as f:
            data = f.read()
        sha256 = hashlib.sha256(data).hexdigest()

        # JSON 로드
        json_path = f"gdrive/My Drive/AI보안교육/part2/vt_json/{sha256}.json"
        with open(json_path, 'r', encoding='utf-8') as f:
            vt_json = json.load(f)

        # 벤더별 집계 변수
        key_cnt_total = 0
        val_cnt_total = 0
        valid_vendor_cnt = 0

        for v in vt_json['data']:
            attrs = v.get('attributes', {})
            # 레지스트리 관련 key 목록
            key_list = attrs.get('registry_keys_opened', [])
            val_list = attrs.get('registry_keys_set', [])

            # 양쪽 다 리스트인 경우에만 집계
            if isinstance(key_list, list) or isinstance(val_list, list):
                valid_vendor_cnt += 1
                key_cnt_total += len(key_list)
                val_cnt_total += len(val_list)

        if valid_vendor_cnt > 0:
            df.loc[sha256, 'registry_open_cnt']   = key_cnt_total / valid_vendor_cnt
            df.loc[sha256, 'registry_set_cnt'] = val_cnt_total / valid_vendor_cnt


In [44]:
df

,target,file_deleted_cnt,registry_open_cnt,registry_set_cnt
hash,,,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,31.0,75.0,9.333333
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,62.0,112.5,125.500000
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,10.0,22.0,2.000000
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,7.0,60.5,1.000000
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,190.0,100.0,6.333333
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,NaN,16.5,0.000000
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,7.0,74.0,1.500000
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,NaN,NaN,NaN
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,81.0,100.0,4.333333


In [45]:
for vendor_idx in range(vt_json['meta']['count']):
    attrs = vt_json['data'][vendor_idx]['attributes']
    # 속성 이름 중에 'registry' 가 들어가는 것만 골라본다
    registry_keys = [k for k in attrs.keys() if 'process' in k.lower()]
    print("==== 벤더 {}: {} ====".format(
        vt_json['data'][vendor_idx]['id'].split('_')[1], registry_keys))

==== 벤더 Microsoft Sysinternals: ['processes_injected', 'processes_tree', 'processes_terminated', 'processes_created'] ====
==== 벤더 ReaQta-Hive: ['processes_terminated', 'processes_created'] ====
==== 벤더 Zenbox: ['processes_tree'] ====


In [46]:
# 1) 결과를 담을 컬럼 추가
df['processes_injected_cnt'] = np.nan
df['processes_terminated_cnt'] = np.nan
df['processes_created_cnt'] = np.nan

for dir_name in os.listdir(colab_path):
    folder = os.path.join(colab_path, dir_name)
    for filename in os.listdir(folder):
        # 파일 해시 계산
        file_path = os.path.join(folder, filename)
        with open(file_path, 'rb') as f:
            data = f.read()
        sha256 = hashlib.sha256(data).hexdigest()

        # JSON 로드
        json_path = f"gdrive/My Drive/AI보안교육/part2/vt_json/{sha256}.json"
        with open(json_path, 'r', encoding='utf-8') as f:
            vt_json = json.load(f)

        # 벤더별 집계 변수
        inj_cnt_total = 0
        ter_cnt_total = 0
        cre_cnt_total = 0
        valid_vendor_cnt = 0

        for v in vt_json['data']:
            attrs = v.get('attributes', {})
            # 레지스트리 관련 key 목록
            inj_list = attrs.get('processes_injected', [])
            ter_list = attrs.get('processes_terminated', [])
            cre_list = attrs.get('processes_created', [])

            # 양쪽 다 리스트인 경우에만 집계
            if isinstance(inj_list, list) or isinstance(ter_list, list) or isinstance(cre_list, list):
                valid_vendor_cnt += 1
                inj_cnt_total += len(inj_list)
                ter_cnt_total += len(ter_list)
                cre_cnt_total += len(cre_list)

        if valid_vendor_cnt > 0:
            df.loc[sha256, 'processes_injected_cnt']   = inj_cnt_total / valid_vendor_cnt
            df.loc[sha256, 'processes_terminated_cnt'] = ter_cnt_total / valid_vendor_cnt
            df.loc[sha256, 'processes_created_cnt'] = cre_cnt_total / valid_vendor_cnt


In [47]:
df

,target,file_deleted_cnt,registry_open_cnt,registry_set_cnt,processes_injected_cnt,processes_terminated_cnt,processes_created_cnt
hash,,,,,,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,31.0,75.0,9.333333,0.333333,0.333333,0.666667
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,62.0,112.5,125.500000,0.500000,1.000000,1.000000
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,10.0,22.0,2.000000,0.500000,1.500000,1.000000
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,7.0,60.5,1.000000,0.000000,1.500000,1.000000
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,190.0,100.0,6.333333,0.333333,0.666667,1.000000
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,NaN,16.5,0.000000,0.000000,0.500000,0.500000
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,7.0,74.0,1.500000,0.000000,0.500000,1.000000
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,NaN,NaN,NaN,NaN,NaN,NaN
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,81.0,100.0,4.333333,0.333333,1.333333,1.000000


In [48]:
# 결과를 담을 컬럼들 미리 생성
df['process_cnt_avg'] = np.nan

for dir_name in os.listdir(colab_path):
    folder = os.path.join(colab_path, dir_name)
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)

        # SHA256 해시 계산
        with open(file_path, 'rb') as f:
            sha256 = hashlib.sha256(f.read()).hexdigest()

        # VT JSON 로드
        json_path = f"gdrive/My Drive/AI보안교육/part2/vt_json/{sha256}.json"
        with open(json_path, 'r', encoding='utf-8') as f:
            vt_json = json.load(f)

        total_counts = 0
        vendor_with_data = 0

        # 각 벤더별 attributes 검사
        for v in vt_json.get('data', []):
            attrs = v.get('attributes', {})

            # "process"가 이름에 들어가고, 값이 리스트인 필드만 골라서 개수 합산
            process_fields = [k for k,val in attrs.items()
                              if 'process' in k.lower() and isinstance(val, list)]
            if not process_fields:
                continue

            # 벤더별 총 프로세스 활동 개수
            cnt = sum(len(attrs[k]) for k in process_fields)
            total_counts += cnt
            vendor_with_data += 1

        # 평균값 계산 후 DataFrame에 기록
        if vendor_with_data > 0:
            df.loc[sha256, 'process_cnt_avg'] = total_counts / vendor_with_data

In [49]:
df

,target,file_deleted_cnt,registry_open_cnt,registry_set_cnt,processes_injected_cnt,processes_terminated_cnt,processes_created_cnt,process_cnt_avg
hash,,,,,,,,
d0ceb18272966ab62b8edff100e9b4a6a3cb5dc0f2a32b2b18721fea2d9c09a5,1,31.0,75.0,9.333333,0.333333,0.333333,0.666667,3.500000
96cdfd7b263947a6a7c0db54141a6b8d7777db0a03a17cbf95666d98422f937b,1,62.0,112.5,125.500000,0.500000,1.000000,1.000000,6.000000
5a780d6630639ffb7fd3d295c182eaa2a7cad2c70248c5ba8f334bb3803353ca,1,10.0,22.0,2.000000,0.500000,1.500000,1.000000,4.000000
f5b4fb293ada97a8c4b92bb380747da1d71336f144332c95ddef3f8522f7ce87,1,7.0,60.5,1.000000,0.000000,1.500000,1.000000,3.500000
908b64b1971a979c7e3e8ce4621945cba84854cb98d76367b791a6e22b5f6d53,1,190.0,100.0,6.333333,0.333333,0.666667,1.000000,2.666667
f1d62648ef915d85cb4fc140359e925395d315c70f3566b63bb3e21151cb2ce3,0,NaN,16.5,0.000000,0.000000,0.500000,0.500000,2.000000
6635e48ad3f02fe5cac9669f974d27ec2fa77b1f6b519f09616630b6531a6ca8,0,7.0,74.0,1.500000,0.000000,0.500000,1.000000,3.000000
f1e3d8851c7b41e3838eae69a95224c92abd9a33117238ef9730233b5b14cdc2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58189cbd4e6dc0c7d8e66b6a6f75652fc9f4afc7ce0eba7d67d8c3feb0d5381f,0,81.0,100.0,4.333333,0.333333,1.333333,1.000000,3.666667
